In [1]:
import numpy as np
import pandas as pd


In [2]:
df=pd.read_csv("C:\\Users\\Jarvis\\Downloads\\fraud.csv")


In [4]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [3]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [6]:
print(df.type.value_counts())

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64


In [18]:
type = df["type"].value_counts()
transactions = type.index
quantity = type.values


import plotly.express as px
figure = px.pie(df,
               values=quantity,
               names=transactions,width=400,hole =0.4,
               title= "Distribution of Transaction Type")
figure.show()

Now let's have a look at the correlation between the features of the 
data with the isFraud column:

In [21]:
# cahcking correlation
correlation = df.corr()
print(correlation['isFraud'].sort_values(ascending=False))

C:\Users\Jarvis\AppData\Local\Temp\ipykernel_23824\319734780.py:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



isFraud           1.000000
amount            0.076688
isFlaggedFraud    0.044109
step              0.031578
oldbalanceOrg     0.010154
newbalanceDest    0.000535
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
Name: isFraud, dtype: float64


Now let's transform the categorical features into numerical .Here i will also 
transform the values of the isFraoud column into no Fraud lavels to have a
better understandion of the output;

In [22]:
df['type']= df['type'].map({"CASH_OUT":1 , "PAYMENT":2,
                           "CASH_IN":3 , "TRANSFER":4,
                           "DEBIT":5 })
df["isFraud"]= df["isFraud"].map({0: "No Fraud",1:"Fraud"})
print(df.head())

   step  type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1     2   9839.64  C1231006815       170136.0       160296.36   
1     1     2   1864.28  C1666544295        21249.0        19384.72   
2     1     4    181.00  C1305486145          181.0            0.00   
3     1     1    181.00   C840083671          181.0            0.00   
4     1     2  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest   isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0  No Fraud               0  
1  M2044282225             0.0             0.0  No Fraud               0  
2   C553264065             0.0             0.0     Fraud               0  
3    C38997010         21182.0             0.0     Fraud               0  
4  M1230701703             0.0             0.0  No Fraud               0  


online Payment Fraud Dection Model

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            float64
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         object 
 10  isFlaggedFraud  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 534.0+ MB


In [23]:
from sklearn.model_selection import train_test_split
x=np.array(df[["type" , "amount","oldbalanceOrg","newbalanceOrig"]])
y=np.array(df[["isFraud"]])

In [24]:
# training  a machine learning model
from sklearn.tree import DecisionTreeClassifier
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.10,random_state=23)
model= DecisionTreeClassifier()
model.fit(x_train,y_train)
print(model.score(x_test,y_test))

0.9997029525572798


In [21]:
x_train.shape , x_test.shape , y_train.shape , y_test.shape

((5726358, 4), (636262, 4), (5726358, 1), (636262, 1))

Now lets classify whether a transactionn is a fraud or not  by feeding
about a transaction into the model

In [32]:
# feature = [type, amount , oldbalanceOrg, newbalanceOrig]
features = np.array([[4,9000.60,9000.60 ,0.0]])
print(model.predict(features))

['Fraud']
